# **MiniHack KeyRoom KB**

**Author**: Riccardo Parente\
**Course**: Artificial Intelligence Fundamentals 2023/2024

### **Introduction**

The goal of this project is to implement a knowledge base written in Prolog and to use it to resolve the puzzle given by the "KeyRoom" environment of the MiniHack Environment Zoo. In this environment the agent must find a key to unlock a door, enter the room and go to the exit.

### **Requirements**
To run the project with its software dependencies on Google Colab (used also for the assessment) the following commands must be runned.

In [ ]:
!sudo apt update
!sudo apt install -y build-essential autoconf libtool pkg-config python3-dev \
    python3-pip python3-numpy git flex bison libbz2-dev

!wget -O - https://apt.kitware.com/keys/kitware-archive-latest.asc 2>/dev/null | sudo apt-key add -
!sudo apt-add-repository 'deb https://apt.kitware.com/ubuntu/ bionic main'
!sudo apt-get update && apt-get --allow-unauthenticated install -y \
    cmake \
    kitware-archive-keyring

!sudo rm $(which cmake)
!$(which cmake) --version

!sudo apt install swi-prolog

!pip3 install -U nle
!pip install minihack
!pip install pyswip

In [2]:
import gym
import minihack
from pyswip import Prolog
import matplotlib.pyplot as plt
import time
import IPython.display as display
from utils import *

### **Knowledge base**
The knowledge base was constructed exploiting the knowledge of the environment, creating an `action(X)` for each possible action X permitted in the environment: in particular, all movement actions were grouped into the two predicates `action(move_towards_goal(Direction))` and `action(move(Direction))` to take advantage of the unification of the direction; the first one was used when the agent knew where the current goal was and the second one when some exploration was needed, since the environment is partially observable. Additionally the actions were ordered by priority.\
The predicate `next_goal(Goal)` unifies the current sub-goal of the puzzle (find the key, open the door, enter the room, escape), using the `has/2`, `is_closed/1` and `has_entered/0` predicates.\
Some knowledge of the NetHack game was also needed to add some logic (`close_direction_door(R, C, D, Direction)`) to enter the room behind the door since in the game a door cannot be traversed diagonally.
To track the structure of the room and the position of every important element of the environment, including the agent, the `position/3` predicate was used. Also, to avoid the agent entering a loop of movements, the `explored/2` predicate was used.

### **Methodology**
Each "run" of the problem (i.e. each time `env.reset()` is called) was limited in the number of steps, to avoid endless runs. Also each time the knowledge base was "cleaned" of all the predicates that were inserted during the execution.\
In each step the first operation is the parsing of the current state (`process_state`) where predicates are inserted in the knowledge base according to the observation received. The knowledge base is then queried for the action to perform and the first action, the one with the most priority, is chosen; finally the action is performed and the step ends.

In [65]:
NUM_EPISODES = 100
is_little = True
def_file = "MiniHack-KeyRoom-S15-v0"
if (is_little):
    def_file = "MiniHack-KeyRoom-S5-v0"
KB = Prolog()
KB.consult("kb.pl")
KB.retractall("has(agent, _)")
KB.retractall("explored(_,_)")
KB.assertz("is_closed(door)")
env = gym.make(def_file, observation_keys=("screen_descriptions","chars", "pixel", "inv_strs", "message"))

In [ ]:
total_reward = 0
total_steps = 0
for episode in range(NUM_EPISODES):
    steps = 0
    reward = 0
    ep_states = []
    state = env.reset()
    ep_states.append(state['pixel'])
    done = False
    while not done and steps < 50:
        process_state(KB, state)
        try:
            action = list(KB.query('action(X)'))[0]['X']
            print(action)
        except Exception as e:
            action = None

        if action:
            state, reward, done, info = env.step(get_action(action))
            if action == "pick":
                KB.assertz("has(agent, key)")
                KB.retractall("explored(_,_)")
            elif action == "open_door":
                KB.retractall("is_closed(_)")
                KB.retractall("explored(_,_)")
            ep_states.append(state['pixel'])
        else:
            print("ERROR: impossible to perform any action.")
            break

        steps += 1

    plot_sequence(ep_states, is_little)
    plt.imshow(ep_states[-1][115:275, 480:750] if is_little else ep_states[-1][50:325, 480:800])
    print(f'Episode {episode} - {steps} steps')
    print(f'Final reward: {reward}')
    time.sleep(0.75)
    KB.retractall("has(agent, _)")
    KB.retractall("explored(_,_)")
    KB.retractall("position(_,_,_)")
    KB.retractall("has_entered(_)")
    KB.assertz("is_closed(door)")
    total_reward += reward
    if (reward != 0):
        total_steps += steps

print(f'After {NUM_EPISODES} episodes, mean reward is {total_reward/NUM_EPISODES}')
if (total_reward != 0):
    print(f'After {NUM_EPISODES} episodes, mean steps is {total_steps/total_reward}')

### **Assessment**
The assessment has been conducted running 100 random generated instances of the four main KeyRoom variants made available by the MiniHack environment.\
The main factor analyzed was the success rate, that is whether the agent reached the final goal in less than 50 steps, that was considered a reasonable number for each variation of the problem. Also the mean steps needed to reach the goal were taken in consideration for comparison between the variants.\
The table below reports a summary of the results:
|                                | S5    | S15   | Dark-S5 | Dark-S15 |
|--------------------------------|-------|-------|---------|----------|
| **Mean reward (success rate)** |  0.95 |  0.68 |   0.86  |   0.02   |
| **Mean steps**                 | 11.94 | 26.81 |  15.02  |   39.5   |

The results showed that, as expected, the agent performed better in the smaller environment, even in the "Dark" variant, where the agent sees only the adjacent cells; the reduced observability implied a decrease of the success rate and an increased mean number of steps in the "Dark" variants with respect to the normal counterparts.\
It has been seen that the success rate drops whenever a more "intelligent" exploration of the environment is needed (the exploration embedded in the knowledge base just picks a feasible direction in a clockwise manner).

### **Conclusions**
The project showed that a knowledge based agent is able to successfully complete a task exploiting the informations given by the nature of the environment, but the partial observability and the need for an intelligent form of search limit the efficiency and even the ability to reach the goal.

### **Appendix**

#### **Relationship with the course**
The project focuses on the area of first order logic and (indirectly) inference. In particular the building of a knowledge base from the informations given by the problem/environment and the subsequent exploitation of the knowledge base to resolve a specific problem.

##### **Github**
The GitHub repository can be found at the following link [https://github.com/RiccardoParente/AIF-Project](https://github.com/RiccardoParente/AIF-Project)